<a href="https://colab.research.google.com/github/Abdulmukhtor/ML-Supervised--learning-2-bolim/blob/main/Colaboratory%EC%97%90_%EC%98%A4%EC%8B%A0_%EA%B2%83%EC%9D%84_%ED%99%98%EC%98%81%ED%95%A9%EB%8B%88%EB%8B%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loyihalar tahlili (KLIB orqali)

In [ ]:
!git clone https://github.com/Abdulmukhtor/ML-Supervised--learning-2-bolim/blob/main/1-dars%20vazifa/Bank_Personal_Loan_Modelling.csv


Cloning into 'Bank_Personal_Loan_Modelling.csv'...
fatal: repository 'https://github.com/Abdulmukhtor/ML-Supervised--learning-2-bolim/blob/main/1-dars%20vazifa/Bank_Personal_Loan_Modelling.csv/' not found


In [ ]:
import pandas as pd

# urlda raw yozilishining sababi githubdagi fayllarning asosiy (default) URLsi veb-sahifa  ko'rinishida bo'ladi , bu esa pandas.read_csv()da o'qilamydi
url = "https://raw.githubusercontent.com/Abdulmukhtor/ML-Supervised--learning-2-bolim/main/1-dars%20vazifa/Bank_Personal_Loan_Modelling.csv"
#"Raw" versiya — bu faqat faylning o‘zini ko‘rsatadigan URL. Uni GitHub interfeysidan olish mumkin
df = pd.read_csv(url)  # yangilangan URL
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


KUTUBXONALARNI yuklab olish

In [ ]:
!pip install zipcodes
!pip install pgeocode
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pgeocode
from sklearn.model_selection import train_test_split, KFold, cross_val_score , GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report,confusion_matrix,precision_score,jaccard_score,recall_score,f1_score,fbeta_score, make_scorer,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE , ADASYN
from imblearn.under_sampling import TomekLinks , EditedNearestNeighbours
from imblearn.pipeline import Pipeline
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.7/726.7 kB 22.2 MB/s eta 0:00:00


Data Pre Processing

In [ ]:
#Ma'lumotlar jadvali tuzish

df = pd.DataFrame(df)
df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [ ]:
# Kearaksiz ustunlarni tashlab yuborish

df.drop(['ID'],axis=1,inplace =True)
df

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [ ]:
# Qayta takrorlangan elementlarni chiqarish

print(len(df[df.duplicated()]))

0


In [ ]:
# tushib qolgan qiymatlarni tekshirish

df.isna().sum()

,0
Age,0
Experience,0
Income,0
ZIP Code,0
Family,0
CCAvg,0
Education,0
Mortgage,0
Personal Loan,0
Securities Account,0


In [ ]:
df.isnull().sum()

,0
Age,0
Experience,0
Income,0
ZIP Code,0
Family,0
CCAvg,0
Education,0
Mortgage,0
Personal Loan,0
Securities Account,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 5000 non-null   int64  
 1   Experience          5000 non-null   int64  
 2   Income              5000 non-null   int64  
 3   ZIP Code            5000 non-null   int64  
 4   Family              5000 non-null   int64  
 5   CCAvg               5000 non-null   float64
 6   Education           5000 non-null   int64  
 7   Mortgage            5000 non-null   int64  
 8   Personal Loan       5000 non-null   int64  
 9   Securities Account  5000 non-null   int64  
 10  CD Account          5000 non-null   int64  
 11  Online              5000 non-null   int64  
 12  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 507.9 KB


In [ ]:
# dataset haqida umumiy ma'lumotni chiqarish
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,5000.0,45.338400,11.463166,23.0,35.0,45.0,55.0,67.0
Experience,5000.0,20.104600,11.467954,-3.0,10.0,20.0,30.0,43.0
Income,5000.0,73.774200,46.033729,8.0,39.0,64.0,98.0,224.0
ZIP Code,5000.0,93152.503000,2121.852197,9307.0,91911.0,93437.0,94608.0,96651.0
Family,5000.0,2.396400,1.147663,1.0,1.0,2.0,3.0,4.0
CCAvg,5000.0,1.937938,1.747659,0.0,0.7,1.5,2.5,10.0
Education,5000.0,1.881000,0.839869,1.0,1.0,2.0,3.0,3.0
Mortgage,5000.0,56.498800,101.713802,0.0,0.0,0.0,101.0,635.0
Personal Loan,5000.0,0.096000,0.294621,0.0,0.0,0.0,0.0,1.0
Securities Account,5000.0,0.104400,0.305809,0.0,0.0,0.0,0.0,1.0


In [ ]:
# biz bu yerda "Experience" ustunida "min" qiymatida manfiy qiymat borligini ko'rishimiz mumkin
# biz buni numpy kutubxonasining "abs" funksiyasi orqali musbat qiymatga aylantiramiz, chunki tajriba doimo o'suvchi musbat qiymat

df['Experience'] = df['Experience'].abs()

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,5000.0,45.338400,11.463166,23.0,35.0,45.0,55.0,67.0
Experience,5000.0,20.134600,11.415189,0.0,10.0,20.0,30.0,43.0
Income,5000.0,73.774200,46.033729,8.0,39.0,64.0,98.0,224.0
ZIP Code,5000.0,93152.503000,2121.852197,9307.0,91911.0,93437.0,94608.0,96651.0
Family,5000.0,2.396400,1.147663,1.0,1.0,2.0,3.0,4.0
CCAvg,5000.0,1.937938,1.747659,0.0,0.7,1.5,2.5,10.0
Education,5000.0,1.881000,0.839869,1.0,1.0,2.0,3.0,3.0
Mortgage,5000.0,56.498800,101.713802,0.0,0.0,0.0,101.0,635.0
Personal Loan,5000.0,0.096000,0.294621,0.0,0.0,0.0,0.0,1.0
Securities Account,5000.0,0.104400,0.305809,0.0,0.0,0.0,0.0,1.0


In [ ]:
# "Income" ustuni kabi yillik daromadga ega bo'lish uchun "CCAvg"ni 12ga ko'paytiramiz

df['CCAvg']=df['CCAvg'].astype(float)*12
df

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,91107,4,19.2,1,0,0,1,0,0,0
1,45,19,34,90089,3,18.0,1,0,0,1,0,0,0
2,39,15,11,94720,1,12.0,1,0,0,0,0,0,0
3,35,9,100,94112,1,32.4,2,0,0,0,0,0,0
4,35,8,45,91330,4,12.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,92697,1,22.8,3,0,0,0,0,1,0
4996,30,4,15,92037,4,4.8,1,85,0,0,0,1,0
4997,63,39,24,93023,2,3.6,3,0,0,0,0,0,0
4998,65,40,49,90034,3,6.0,2,0,0,0,0,1,0


In [ ]:
discrete = ['Family' , 'Education' ,'Personal Loan' , 'Securities Account' , 'CD Account' , 'Online' , 'CreditCard']
continuous = ['Age' , 'Experience', 'Income', 'CCAvg' , 'Mortgage']

In [ ]:
# aniq, sanaladigan ustunlar qiymatlari

for i in discrete:
  print(f'{i}:{df[i].unique()}')

Family:[4 3 1 2]
Education:[1 2 3]
Personal Loan:[0 1]
Securities Account:[1 0]
CD Account:[0 1]
Online:[0 1]
CreditCard:[0 1]


klib

In [ ]:
!pip install klib
import klib

In [ ]:
# datasetimizda 5000 qator, 13ta ustun bor
# duplicated elementlar yo'q
# missing valuelar yo'q
# memory usage 80%ga kamaygan
# 1.Klib qilishda agar missing valuelarni tashlab yuborishda bizda keraksiz, muhim bo'lmagan qatorlar bo'lsa,
# missing value handle qilishdagi kabi dataset katta bo'lsa, uncha muhim bo'lmagan qatorlarni drop qilib yuboradi
# 2.Ma'lumotlarni kompyuterga saqlash usuli bilan (data structure) bilan memory usagelarini kam olyapti
# 3.Katta harflarni kichik harflar bilan olyapti
# 4.Ustun nomlari orasidagi bo'sh joylarni olib tashlashyapti

df=klib.data_cleaning(df)

Shape of cleaned data: (5000, 13) - Remaining NAs: 0


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.4 MB (-80.0%)



In [ ]:
# memory usage 507.2 kbdan 102.7 kbga kamaygan
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   age                 5000 non-null   int8   
 1   experience          5000 non-null   int8   
 2   income              5000 non-null   int16  
 3   zip_code            5000 non-null   int32  
 4   family              5000 non-null   int8   
 5   ccavg               5000 non-null   float32
 6   education           5000 non-null   int8   
 7   mortgage            5000 non-null   int16  
 8   personal_loan       5000 non-null   int8   
 9   securities_account  5000 non-null   int8   
 10  cd_account          5000 non-null   int8   
 11  online              5000 non-null   int8   
 12  credit_card         5000 non-null   int8   
dtypes: float32(1), int16(2), int32(1), int8(9)
memory usage: 102.7 KB


In [ ]:
print(df["income"].describe()) # Min va max qiymatlarni ko‘rish

count    5000.000000
mean       73.774200
std        46.033729
min         8.000000
25%        39.000000
50%        64.000000
75%        98.000000
max       224.000000
Name: income, dtype: float64


In [ ]:


print(df["income"].dtype)  # Hozirgi dtype ni tekshirish

print(df["income"].isna().sum())  # NaN qiymatlar bor yoki yo‘qligini tekshirish

print(df["income"].unique())  # Ustundagi barcha takrorlanmas qiymatlarni chiqarish

int16
0
[ 49  34  11 100  45  29  72  22  81 180 105 114  40 112 130 193  21  25
  63  62  43 152  83 158  48 119  35  41  18  50 121  71 141  80  84  60
 132 104  52 194   8 131 190  44 139  93 188  39 125  32  20 115  69  85
 135  12 133  19  82 109  42  78  51 113 118  64 161  94  15  74  30  38
   9  92  61  73  70 149  98 128  31  58  54 124 163  24  79 134  23  13
 138 171 168  65  10 148 159 169 144 165  59  68  91 172  55 155  53  89
  28  75 170 120  99 111  33 129 122 150 195 110 101 191 140 153 173 174
  90 179 145 200 183 182  88 160 205 164  14 175 103 108 185 204 154 102
 192 202 162 142  95 184 181 143 123 178 198 201 203 189 151 199 224 218]
